In [1]:
from IPython.display import display
from torchnlp.datasets import iwslt_dataset
from torchnlp.word_to_vector import GloVe
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from utils import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
vectors = GloVe()

In [3]:
train_raw = [sentence['en'] for sentence in iwslt_dataset(train=True)]
dev_raw = [sentence['en'] for sentence in iwslt_dataset(dev=True)]
test_raw = [sentence['en'] for sentence in iwslt_dataset(test=True)]
print("Train:", len(train_raw))
print("Dev:", len(dev_raw))
print("Test:", len(test_raw))

Train: 196884
Dev: 993
Test: 1305


In [4]:
print("Preprocessing the train data")
train_x, train_y = preprocess_data(train_raw, vectors)
print("Preprocessing the dev data")
dev_x, dev_y = preprocess_data(dev_raw, vectors)
print("Preprocessing the test data")
test_x, test_y = preprocess_data(test_raw, vectors)

Preprocessing the train data
Tokenizing:



Padding:



Creating labels:



Get word vector weights



Converting labels to tensor


...


Padding:



Creating labels:



Get word vector weights



Converting labels to tensor



Printing samples of the dataset
['<pad>', '<pad>', 'When', 'I', 'was'] ('o', True)
['<pad>', 'When', 'I', 'was', 'in'] ('o', True)
['When', 'I', 'was', 'in', 'my'] ('o', False)
['I', 'was', 'in', 'my', '20s'] ('o', False)
['was', 'in', 'my', '20s', 'I'] ('o', False)
['in', 'my', '20s', 'I', 'saw'] (',', False)
['my', '20s', 'I', 'saw', 'my'] ('o', True)
['20s', 'I', 'saw', 'my', 'very'] ('o', False)
['I', 'saw', 'my', 'very', 'first'] ('o', False)
['saw', 'my', 'very', 'first', 'psychotherapy'] ('o', False)
['my', 'very', 'first', 'psychotherapy', 'client'] ('o', False)
['very', 'first', 'psychotherapy', 'client', '<pad>'] ('o', False)
['first', 'psychotherapy', 'client', '<pad>', '<pad>'] ('.', False)
['<pad>', '<pad>', 'I', 'was', 'a'] ('o', True)
['<pad>', 'I', 'was', 'a', 'Ph.D.'] ('o', False)
Total number of samples:  21760


In [5]:
from models import PuncDataset
train_loader = DataLoader(PuncDataset(train_x, train_y), batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=0)
dev_loader = DataLoader(PuncDataset(dev_x, dev_y), batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=0)
test_loader = DataLoader(PuncDataset(test_x, test_y), batch_size=BATCH_SIZE,
                         shuffle=True, num_workers=0)

In [6]:
from models import PuncLstm

model = PuncLstm(CLASSES, WINDOW_SIZE)
model.to(device)
criterions = [nn.CrossEntropyLoss(), nn.CrossEntropyLoss()]
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
best_model = None

In [7]:
if __name__=='__main__':
    best_model = train(train_loader, dev_loader, model, criterions, optimizer, 100, device)

    print('Finished Training')

----------------------------
Epoch: 0


Training loss for punctuation: 52.57996456065494
Training loss for capitalization: 20.38393548147198
Validation accuracy: punctuation: 94.4762%, capitalization: 98.564%
Find a new best model!
----------------------------
Epoch: 1


Training loss for punctuation: 42.90522576969511
Training loss for capitalization: 14.358167224240166
Validation accuracy: punctuation: 94.8118%, capitalization: 98.6301%
Find a new best model!
----------------------------
Epoch: 2


Training loss for punctuation: 40.283745361394296
Training loss for capitalization: 13.054546028309915
Validation accuracy: punctuation: 95.0099%, capitalization: 98.6521%
Find a new best model!
----------------------------
Epoch: 3


Training loss for punctuation: 38.489875822755565
Training loss for capitalization: 12.089506015266966
Validation accuracy: punctuation: 94.9439%, capitalization: 98.6851%
----------------------------
Epoch: 4


...

Training loss for punctuation: 33.683906736013626
Training loss for capitalization: 9.38069772382142
Validation accuracy: punctuation: 95.1529%, capitalization: 98.5585%
----------------------------
Epoch: 8


Training loss for punctuation: 32.76949356093961
Training loss for capitalization: 8.848108127943194
Validation accuracy: punctuation: 95.0264%, capitalization: 98.5585%
----------------------------
Epoch: 9


Training loss for punctuation: 31.838123109294056
Training loss for capitalization: 8.352086054951098
Validation accuracy: punctuation: 95.0319%, capitalization: 98.553%
----------------------------
Epoch: 10


Training loss for punctuation: 31.029941956330692
Training loss for capitalization: 7.913224732281793
Validation accuracy: punctuation: 95.1419%, capitalization: 98.52%
----------------------------
Epoch: 11


Training loss for punctuation: 30.289371945587096
Training loss for capitalization: 7.482485956459138
Validation accuracy: punctuation: 95.1695%, capitalization: 98.6026%
Finished Training


In [8]:
dfs = test(test_loader, best_model, device)
display(dfs[0])
display(dfs[1])

..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, po

..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, po

..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
..\aten\src\ATen\native\cudnn\RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, po

Test accuracy: punctuation: 95.2895%, capitalization: 98.6305%


,predicted,predicted correctly,predicted expectation,precision,recall,f_score
punctuation,,,,,,
o,19277,18722,19042,0.971159,0.983143,0.977114
",",1174,826,1354,0.702979,0.609594,0.652964
.,1288,1172,1223,0.909232,0.957516,0.932750
?,21,15,128,0.681818,0.116279,0.198675
!,0,0,13,0.000000,0.000000,0.000000


,predicted,predicted correctly,predicted expectation,precision,recall,f_score
capitalization,,,,,,
No,19299,19101,19201,0.989689,0.994740,0.992208
Yes,2461,2361,2559,0.958976,0.922266,0.940263


In [9]:
#save_model(model)